In [10]:
import sys
sys.path.append("../../")
from cell_builder import sst_seg_cell_builder

from act.cell_model import ACTCellModel
from act.simulator import ACTSimulator
from act.types import SimulationParameters, ConstantCurrentInjection
from act.passive import ACTPassiveModule
from act.data_processing import get_summary_features

import matplotlib.pyplot as plt
import numpy as np

In [2]:
seg_cell = ACTCellModel(
    cell_name = None,
    path_to_hoc_file = None,
    path_to_mod_files = "../../data/SST/seg/modfiles/",
    passive = ["g_pas", "e_pas", "gbar_Ih"],
    active_channels = ["gbar_Nap", "gbar_Im_v2", "gbar_K_T", "gbar_NaTa", "gbar_Kd", "gbar_Ca_LVA", "gbar_Ca_HVA", "gbar_Kv2like", "gbar_Kv3_1"]
)

seg_cell.set_custom_cell_builder(sst_seg_cell_builder)

sim_params = SimulationParameters(
    sim_name = "SST_seg", 
    sim_idx = 1, 
    h_celsius = 37,
    verbose = True, 
    CI = [ConstantCurrentInjection(amp = -0.1, dur = 200, delay = 100)])

simulator = ACTSimulator(output_folder_name = "output")
simulator.submit_job(
    seg_cell,
    sim_params
)
simulator.run_jobs(1)


        ACTSimulator (2025)
        ----------
        When submitting multiple jobs, note that the cells must share modfiles.
        
warning ./Sst-IRES-Cre_Ai14-188740.03.02.01_657210399_m.swc: more than one tree:
  root at line 4
  root at line 14504
  root at line 14725
  root at line 14798
  root at line 15141
  root at line 15144
  root at line 15156
  root at line 15211
  root at line 15318
  root at line 15496
  root at line 15811
  root at line 15857
  root at line 15883
  root at line 16161
  root at line 16200
  root at line 16210
  root at line 16253
  root at line 16286
  root at line 16301
  root at line 16326
  root at line 16574
  root at line 16613
  root at line 16618
  root at line 16645
  root at line 16695
  root at line 16751
  root at line 16765
  root at line 16768
Soma diam (um): 12.661399841308594
Soma L (um): 12.661407470703125
Soma area (cm2): 5.036323076932052e-06
Total area (cm2): 3.6238296823045575e-05


In [3]:
computed_soma_area = 5.036323076932052e-06 # (cm2)
computed_total_area = 3.6238296823045575e-05 # (cm2)

In [ ]:
# Target (SST from a different mouse)
# https://celltypes.brain-map.org/experiment/electrophysiology/488677994

target_rheobase = 160 # pA
target_tau = 9.2 * 1e-3 # (s)
target_R_in = 137 * 1e6 # Ohm
target_V_rest = -72.9 # mV

I = [150, 200, 250, 300, 350]
F = [0, 18, 39, 60, 70]

In [7]:
ACTPassiveModule.compute_spp(target_R_in, computed_total_area, target_tau, target_V_rest)

SettablePassiveProperties(Cm=1.8531026720004962, g_bar_leak=0.0002014242034783148, e_rev_leak=-72.9, g_bar_h=None)

In [ ]:
# Target summary features
sf = get_summary_features(np.zeros((5, 10)), np.zeros((5, 10)))
sf['spike_frequency'] = [0, 18, 39, 60, 70]
sf['mean_i'] = [0.150, 0.200, 0.250, 0.300, 0.350]

for f in ["mean_v", "std_v", "max_ampl_v", "n_spikes", "main_freq", "std_i"]:
    sf[f] = np.nan
sf.to_csv("target_sf.csv", index = False)